In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.vision import *
from fastai.callbacks.hooks import *
from fastai.utils.mem import *


from PIL import Image
from numpy import array

import time

import cv2
import numpy as np

from torchvision.utils import save_image
import torch

In [3]:
def segs2onehot(frame, nchannels):
    """
    Converts a tensor of shape (1, imgx, imgy)
    with discrete values in the range(nchannels)
    to a onehot style tensor of shape (nchannels, imgx, imgy)
    """
    *_, nx, ny = frame.shape
    x,y = np.meshgrid(np.arange(nx), np.arange(ny))
    res = np.zeros((nchannels, nx, ny))
    res[frame.T.ravel(), x.ravel(), y.ravel()] = 1
    return res

In [20]:
segment_count = 20000
fileCount = 21

vector_list = []
mag_ang_list_one = []

In [21]:
# Open the optical flow matrices
matrices = np.load('vectorFolder/optFlowMat21.npy')
len(matrices)

400

In [22]:
for matrix in matrices:
    
    # Segmentation: Load in segmentation image
    seg_im = cv2.imread(f'/home/jupyter/.fastai/data/camvid/segmentedImages/segImage{segment_count}.png')
    seg = seg_im[:,:,0]
    
    # Segmentation: Perform One-Hot Encoding
    segOH = segs2onehot(seg, 32)
    
    # Segmentation: Change dimensions # 01 x 32 x 360 x 480
    seg1 = segOH[None, :,:,:]
        
    # OPTICAL FLOW: Change dimensions # 02 x 01 x 360 x 480
    optFlow1 = matrix[:, None, :, :]
    
    # Dot product for optFlow1 * seg1
    vector = (optFlow1 * seg1)
    
    # Average Pooling: transpose to prepare vector
    vect1 = vector.T
    
    # Average Pooling: Dimensions should be 32 x 2
    ave1 = np.mean(vect1, axis=1)
    ave2 = np.mean(ave1, axis=0)
    
    vector_list.append(ave2)
    
    segment_count += 1
    
np.save(f'vectorFolder/vectorList{fileCount}.npy', vector_list)
vector_list.clear()

In [23]:
print(segment_count)

20400


# Alt script below

In [4]:
vector_list = []
count = 14001
# Save your vectors using this variable
fileCount = 28

# magAngList = []
mag_ang_list_one = []

In [5]:
cap = cv2.imread('/home/jupyter/.fastai/data/camvid/commaImages/frame14000.jpg')
frame1 = cap
prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

In [6]:
# Use this to reset optical flow in between 5,000 frames 


# frame1 = cv2.imread(f'/home/jupyter/.fastai/data/camvid/commaImages/frame{count}.jpg')
# prvs = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
# count += 1

In [7]:
while (count < 20401):
    frame2 = cv2.imread(f'/home/jupyter/.fastai/data/camvid/commaImages/frame{count}.jpg')
    nxt = cv2.cvtColor(frame2,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,nxt, None, 0.5, 3, 15, 3, 5, 1.2, 0)

    # Optical Flow: set up and document magnitude, angle
    mag, ang = (flow[...,0], flow[...,1])
    
    # Optical Flow: Combine mag, ang
    magAng = np.stack([mag, ang])
    
#     magAngList.append(magAng)
    mag_ang_list_one.append(magAng)
    
    # Segmentation: Load in segmentation image
    seg_im = cv2.imread(f'/home/jupyter/.fastai/data/camvid/segmentedImages/segImage{count}.png')
    seg = seg_im[:,:,0]
    
    # Segmentation: Perform One-Hot Encoding
    segOH = segs2onehot(seg, 32)
    
    # Segmentation: Change dimensions # 01 x 32 x 360 x 480
    seg1 = segOH[None, :,:,:]

    # Optical Flow: Isolate the most recent optical flow matrix 
    optFlow = mag_ang_list_one[0]
    
    
    # Optical Flow: Change dimensions # 02 x 01 x 360 x 480
    optFlow1 = optFlow[:, None, :, :]
    
    
    # Dot product for optFlow1 * seg1
    vector = (optFlow1 * seg1)
    
    # Average Pooling: transpose to prepare vector
    vect1 = vector.T
    vect2 = vect1.T
    
    # Average Pooling: Dimensions should be 32 x 2
    ave1 = np.mean(vect2, axis=1)
    ave2 = np.mean(ave1, axis=0)
    
    vector_list.append(ave2)

    # Empty mag_ang_list_one list 
    mag_ang_list_one.clear()
        
    # Optical Flow: on to the next images 
    prvs = nxt
    count += 1
    
    # Print Status of count 
    if count % 1000 == 0 or count % 500 == 0:
        print("count = ", count)
        
    if count % 500 == 0:
        np.save(f'vectorFolder/vectorList{fileCount}.npy', vector_list)
        fileCount += 1
        vector_list.clear()

cv2.destroyAllWindows()
print(count)

count =  14500
count =  15000
count =  15500
count =  16000
count =  16500
count =  17000
count =  17500
count =  18000
count =  18500
count =  19000
count =  19500
count =  20000
20401


In [8]:
print(count)

20401


In [9]:
# Save the final vectors 
np.save(f'vectorList_{fileCount + 1}.npy', vector_list)